In [1]:
import numpy as np
import sys
from glob import glob
from os import path

import ray
from ray.tune import Trainable, Tuner
from ray.tune.registry import register_trainable, validate_trainable
from ray.rllib.core.rl_module.rl_module import RLModuleSpec
from ray.rllib.core.rl_module.multi_rl_module import MultiRLModuleSpec
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.rllib.policy.policy import Policy
from ray.tune.registry import get_trainable_cls, register_env

from pettingzoo.sisl import waterworld_v4

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-24 23:02:18,667	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-24 23:02:19,085	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
parser = add_rllib_example_script_args(
    default_iters=10,
    default_timesteps=1000000,
    default_reward=300,
)
args = parser.parse_args(args=[])
args.num_env_runners = 10
args.env = 'waterworld'
args.algo = 'PPO'
args.num_agents = 4
args.test_agents = 4

register_env(f"{args.num_agents}_agent_env", lambda _: ParallelPettingZooEnv(waterworld_v4.parallel_env(n_pursuers=args.num_agents)))
policies = {f"pursuer_{i}" for i in range(args.num_agents)}

config = (
    get_trainable_cls("PPO")
    .get_default_config()
    .environment(f"{args.num_agents}_agent_env")
    .multi_agent(
        policies=policies,
        policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
    )
    .rl_module(
        rl_module_spec=MultiRLModuleSpec(
            rl_module_specs={p: RLModuleSpec() for p in policies},
        ),
    )
)

algo = config.build()

/opt/conda/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/opt/conda/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/opt/conda/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
2024-12-24 23:02:33,010	WARNING algorithm_config.py:4415 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.
/opt/conda/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py

In [20]:
#algo.config.environment
#algo.remove_policy(f'{env.agent_name}_{i}')
#algo.add_policy(f'{env.agent_name}_{i}', policy=new_pols[i])
#algo.get_policy("pursuer_0")
#algo.config.environment()
#algo.env_runner_group.foreach_worker(func=print)
#algo.env_runner.config.environment()
dir(algo.config)
type(algo.config)
for key in algo.config.keys():
    print(f"{key:<40}: {type(key)}")

extra_python_environs_for_driver        : <class 'str'>
extra_python_environs_for_worker        : <class 'str'>
placement_strategy                      : <class 'str'>
num_gpus                                : <class 'str'>
_fake_gpus                              : <class 'str'>
num_cpus_for_main_process               : <class 'str'>
eager_tracing                           : <class 'str'>
eager_max_retraces                      : <class 'str'>
tf_session_args                         : <class 'str'>
local_tf_session_args                   : <class 'str'>
torch_compile_learner                   : <class 'str'>
torch_compile_learner_what_to_compile   : <class 'str'>
torch_compile_learner_dynamo_backend    : <class 'str'>
torch_compile_learner_dynamo_mode       : <class 'str'>
torch_compile_worker                    : <class 'str'>
torch_compile_worker_dynamo_backend     : <class 'str'>
torch_compile_worker_dynamo_mode        : <class 'str'>
torch_ddp_kwargs                        : <class

In [22]:
#dir(config)
#dir(algo)
#dir(algo.env_runner)
#type(algo.env_runner.make_env())

from copy import copy

algo = config.build()
print(len(algo.config.keys()))

config_alt = copy(config)
config_alt["___test_key___"] = 'test_value'

algo2 = config_alt.build()
print(len(algo2.config.keys()))


2024-12-24 23:24:39,116	WARNING algorithm_config.py:4415 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.
/opt/conda/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:568: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/opt/conda/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.

201


2024-12-24 23:24:47,356	WARNING algorithm_config.py:4415 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.
2024-12-24 23:24:47,357	WARNING util.py:61 -- Install gputil for GPU system monitoring.


202


In [46]:
%%bash
#for i in {2..8}; do
#for i in {2,4,6,8}; do
a=(a b c)
for i in "${!a[@]}"; do
    echo $i
done

0
1
2


In [42]:
import numpy as np

# Slice Test
a = [_ for _ in range(1,5)]
print(a)
print("---")
b = np.random.choice(a[1:-1], 3)
print([a[0],*b,a[-1]])

[1, 2, 3, 4]
---
[1, 3, 2, 3, 4]


In [27]:
import lbforaging
import gymnasium as gym

env = gym.make("Foraging-8x8-2p-1f-v3")

/opt/conda/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [8]:
from pettingzoo.sisl import pursuit_v4
penv = pursuit_v4.parallel_env()
penv.reset()

({'pursuer_0': array([[[1., 0., 0.],
          [1., 0., 0.],
          [1., 0., 0.],
          [0., 0., 0.],
          [0., 0., 1.],
          [0., 0., 0.],
          [0., 0., 0.]],
  
         [[1., 0., 0.],
          [1., 0., 0.],
          [1., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
  
         [[1., 0., 0.],
          [1., 0., 0.],
          [1., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
  
         [[1., 0., 0.],
          [1., 0., 0.],
          [1., 0., 0.],
          [0., 1., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
  
         [[1., 0., 0.],
          [1., 0., 0.],
          [1., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
  
         [[1., 0., 0.],
          [1., 0., 0.],
          [1., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
       

In [12]:
dir(penv)
#penv.agents

['__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_is_protocol',
 'action_space',
 'action_spaces',
 'aec_env',
 'agents',
 'close',
 'max_num_agents',
 'metadata',
 'num_agents',
 'observation_space',
 'observation_spaces',
 'possible_agents',
 'render',
 'render_mode',
 'reset',
 'state',
 'step',
 'unwrapped']

In [6]:
#dir(env.env.env.players)
dir(env)
env.reset()


((array([5., 3., 1., 4., 0., 1., 0., 2., 1.], dtype=float32),
  array([5., 3., 1., 0., 2., 1., 4., 0., 1.], dtype=float32)),
 {})

In [7]:
dir(env)

['__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_action_space',
 '_cached_spec',
 '_disable_render_order_enforcing',
 '_has_reset',
 '_is_protocol',
 '_metadata',
 '_np_random',
 '_observation_space',
 '_reward_range',
 '_saved_kwargs',
 'action_space',
 'class_name',
 'close',
 'env',
 'has_reset',
 'metadata',
 'np_random',
 'observation_space',
 'render',
 'render_mode',
 'reset',
 'reward_range',
 'spec',
 'step',
 'unwrapped',
 'wrapper_spec']

In [24]:
out = env.step((1,1))

/opt/conda/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:275: UserWarning: WARN: The reward returned by `step()` must be a float, int, np.integer or np.floating, actual type: <class 'list'>
  logger.warn(


In [25]:
out

((array([6., 5., 1., 0., 0., 2., 5., 7., 2.], dtype=float32),
  array([6., 5., 1., 5., 7., 2., 0., 0., 2.], dtype=float32)),
 [0, 0],
 False,
 False,
 {})

In [55]:
env.reset()

((array([2., 3., 2., 7., 0., 1., 7., 3., 1.], dtype=float32),
  array([2., 3., 2., 7., 3., 1., 7., 0., 1.], dtype=float32)),
 {})

In [56]:
out = env.step((1,1))

/opt/conda/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:275: UserWarning: WARN: The reward returned by `step()` must be a float, int, np.integer or np.floating, actual type: <class 'list'>
  logger.warn(


In [58]:
out

((array([2., 3., 2., 6., 0., 1., 6., 3., 1.], dtype=float32),
  array([2., 3., 2., 6., 3., 1., 6., 0., 1.], dtype=float32)),
 [0, 0],
 False,
 False,
 {})

In [42]:
env.observation_space

Tuple(Box([-1. -1.  0. -1. -1.  0. -1. -1.  0.], [7. 7. 4. 7. 7. 2. 7. 7. 2.], (9,), float32), Box([-1. -1.  0. -1. -1.  0. -1. -1.  0.], [7. 7. 4. 7. 7. 2. 7. 7. 2.], (9,), float32))

In [47]:
import gymnasium as gym

class ForageWrapper():
    def __init__(self, num_agents=2, dim=8):
        import lbforaging
        self.env = gym.make("Foraging-8x8-2p-1f-v3") 
        #self.env = gym.make(f"Foraging-{dim}x{dim}-{num_agents}p-1f-v3") 
        
        self.agents = [f"forager_{i}" for i in range(num_agents)]
        self.num_agents = num_agents
        self.max_num_agents = 9
        self.possible_agents = 9
        self.render_mode = env.render_mode
        self.state = None
        
    def action_spaces(self):
        return dict(zip(self.agents, self.env.action_space))

    def action_space(self, agent):
        idx = self.agents.index(agent)
        return env.action_space[idx]

    def observation_spaces(self):
        return dict(zip(self.agents, self.env.observation_space))
    
    def observation_space(self, agent):
        idx = self.agents.index(agent)
        return env.observation_space[idx]

    def reset(self):
        acts, infos = env.reset()
        return dict(zip(self.agents,acts)), infos

    def step(self, actions):
        acts = actions.values
        out = env.step(acts)
        self.state = dict(zip(self.agents,out[0]))
        out[0] = self.state
        return out

    # Direct Pass
    def render(self): return env.render()
    def close(self): return env.close()
    def unwrapped(self): return env.unwrapped()
    def metadata(self): return env.metadata()

    
    """ LBF base func
    'class_name',
    'has_reset',
    'np_random',
    'reward_range',
    'spec',
    'wrapper_spec'
    """

tenv = ForageWrapper()
dir(tenv)
#tenv.reset()

/opt/conda/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'action_space',
 'action_spaces',
 'agents',
 'close',
 'env',
 'max_num_agents',
 'metadata',
 'num_agents',
 'observation_space',
 'observation_spaces',
 'possible_agents',
 'render',
 'render_mode',
 'reset',
 'state',
 'step',
 'unwrapped']

In [3]:
a = None

if not a:
    print("This")
else:
    print("That")

This
